In [1]:
import pandas as pd
stat_file = pd.DataFrame({'model':[],
                  'model_params':[],
                  'opt_params':[],
                  'run_number':[],
                  'epoch':[],
                  'validation_or_train':[],
                  'accuracy':[],
                  'loss':[],
                'additional_info':[]})
stat_file

,model,model_params,opt_params,run_number,epoch,validation_or_train,accuracy,loss,additional_info


In [2]:
import seaborn as sns 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:

j=pd.read_json('./data.json', lines=True)
j.to_csv('./data.csv')

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation

In [5]:

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/valeriy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/valeriy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/valeriy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/valeriy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/valeriy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [6]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
wnl = WordNetLemmatizer()

def lemmatize_sentence(sent):
    lemmatized = []
    for word, tag in pos_tag(word_tokenize(sent)):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = wnl.lemmatize(word, wntag)

        lemmatized.append(lemma)

    #print(' '.join(lemmatized))
    return ' '.join(lemmatized)
        


In [7]:
import re
from nltk.corpus import stopwords
def clean(row, is_lemmatize):
    
    row = row.lower()
    row = re.sub('[^a-z A-Z 0-9]+', ' ', row)
    row = " ".join([word for word in row.split() if word not in stopwords.words('english')])
    if is_lemmatize:
        row = lemmatize_sentence(row)
    return row

In [8]:
def clean_dataframe(j, is_lemmatize=False):
    j['headline'] = j['headline'].apply(clean,is_lemmatize=is_lemmatize)
    j['short_description'] = j['short_description'].apply(clean,is_lemmatize=is_lemmatize)
    j['link'] = j['link'].apply(clean,is_lemmatize=is_lemmatize)
    j ['authors'] = j['authors'].apply(clean,is_lemmatize=is_lemmatize)
    return j

In [9]:
data_cleaned = clean_dataframe(j,is_lemmatize=True)
data_cleaned.to_csv('data_cleaned_full_lemmatized.csv', index=False)


In [33]:

data_cleaned = pd.read_csv('data_cleaned_not_lemmatized.csv')
data_cleaned.isna().sum()


link                     0
headline                67
category                 0
short_description    20179
authors              37419
date                     0
dtype: int64

Nans are here!!!

In [34]:
columns_to_convert = ['headline', 'short_description', 'authors', 'link']
for col in columns_to_convert:
    data_cleaned[col] = data_cleaned[col].astype(str)


In [35]:
data_cleaned.iloc[1]

link                 https www huffpost com entry american airlines...
headline             american airlines flyer charged banned life pu...
category                                                     U.S. NEWS
short_description    subdued passengers crew fled back aircraft con...
authors                                                 mary papenfuss
date                                                        2022-09-23
Name: 1, dtype: object

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

In [37]:

concat_f = lambda a : ' '.join(a)
grouped = data_cleaned.groupby('category').agg({'headline':concat_f, 'short_description':concat_f, 'authors':concat_f, 'link':concat_f}).reset_index()

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [39]:
le = LabelEncoder()
data_labels = le.fit_transform(data_cleaned.category)
data_labels

array([35, 35,  5, ..., 28, 28, 28])

In [40]:
from sklearn.naive_bayes import ComplementNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score


In [54]:
def run_pipeline(model, data, features=['headline','short_description' ]):
    feature=''
    for el in features:
        feature+=data[el]
    
    x_train, x_test, y_train, y_test = train_test_split(feature,
                                                        data_labels, 
                                                        train_size=0.8, test_size=0.2,
                                                        random_state=42)
    
    
    cv = TfidfVectorizer ()
    X_train = cv.fit_transform(x_train)
    X_test = cv.transform(x_test)


    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    f1, accuracy = f1_score(preds, y_test, average='macro'), accuracy_score(preds, y_test)
    print(f1, accuracy)

    return f1, accuracy


In [55]:
def insert_row(df, row):
    return pd.concat([df, row], ignore_index=True)

In [ ]:
nb = ComplementNB()
feature_list=['headline','short_description' ]
f1, accuracy = run_pipeline(nb, data_cleaned,feature_list)

0.37950420278653324 0.549634897150766


In [60]:
row = {'model': nb.__class__,
        'model_params':None,
        'opt_params':None,
        'run_number':None,
        'epoch':None,
        'validation_or_train':'validation',
        'accuracy':accuracy,
        'loss':None,
          'additional_info':'no lemmatization '+'features: ' + ' '.join(feature_list)}

stat_file = insert_row(stat_file, pd.DataFrame(row, index=[len(stat_file)]))

In [62]:
stat_file.to_csv('MNB-tfidf.csv')